 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
Tangi Migot

In [26]:
import Pkg; Pkg.add("NLPModels")
using ADNLPModels, LinearAlgebra, NLPModels, Printf

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


On pourra trouver de la documentation sur `ADNLPModels` et `NLPModels` ici:
- [juliasmoothoptimizers.github.io/NLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/NLPModels.jl/dev/)
- [juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/)

In [27]:
# Problème test:
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
g(x) = 6 * [x[1]^2 - x[1] - 2*x[1]*x[2] + x[2]^2 + x[2]; -x[1]^2 + 2*x[1]*x[2] + x[1]] # le gradient de f
H(x) = 6 * [2*x[1]-1-2*x[2] -2*x[1]+2*x[2]+1; -2*x[1]+2*x[2]+1 2*x[1]] # la Hessienne de f

H (generic function with 1 method)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

Ci-dessous, vous avez le code de deux fonctions qui ont été vues dans le cours, la recherche linéaire qui satisfait Armijo, et une méthode de Newton avec cette recherche linéaire. Le but de ce laboratoire est d'implémenter d'autres méthodes utiles pour résoudre des problèmes de grandes dimensions.

In [28]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end

armijo (generic function with 1 method)

In [ ]:
#Test pour vérifier que la fonction armijo fonctionne correctement.
using Test #le package Test définit (entre autre) la macro @test qui permet de faire des tests unitaires :-)
xk = ones(2)
gk = g(xk)
dk = - gk
fk = f(xk)
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= fk + 1.0e-4 * t * dot(gk,dk)

xk = [1.5, 0.5]
fk = f(xk)
gk = g(xk)
dk = - gk
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= f(xk) + 1.0e-4 * t * dot(g(xk),dk)


In [ ]:
function newton_armijo(f, g, H, x0; verbose::Bool = true)
  xk  = x0
  fk  = f(xk)
  gk = g(xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    Hk = H(xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

In [ ]:
sol = newton_armijo(f, g, H, [.5, .5])
@test g(sol) ≈ zeros(2) atol = 1.0e-6

On veut améliorer le code de la fonction `newton_armijo` avec les ajouts suivants:
- Changer les paramètre d'entrées de la fonction pour un `nlp`
- Avant d'appeler la recherche linéaire, si `slope = dot(dk, gk)` est plus grand que `-1.0e-4 * norm(dk) * gnorm`, on modifie le système. On fait maximum 5 mise à jour de `λ`, sinon on prend l'opposé du gradient.
```
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
```
Ajouter un compteur sur le nombre de mises à jour de `λ` et ajuster `dk = - gk` si la limite est atteinte.
- On veut aussi détecter et éventuellement arrêter la boucle `while` si la fonction objectif `fk` devient trop petite/négative (inférieure à `-1e15`), i.e. le problème est non-bornée inférieurement.
- On veut ajouter deux critères d'arrêts supplémentaires: 
  - un compteur sur le nombre d'évaluations de f (maximum 1000). Utiliser `neval_obj(nlp)`.
  -  une limite de temps d'execution, `max_time = 60.0`. Utiliser la fonction `time()`.
- Enfin, on voudrait aussi voir un message à l'écran si l'algorithme n'a pas trouvé la solution, i.e. il s'est arrêté à cause de la limite sur le nombre d'itérations, temps, évaluation de fonctions, problème non-borné.

In [29]:
#SOLUTION: fonction à modifier
function newton_armijo(nlp, x0; verbose::Bool = true)
  t1 = time()
  xk  = x0
  # fk  = f(xk)
  fk = obj(nlp,xk)
  # gk = g(xk)
  gk = grad(nlp,xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  critereArreteAtteint = [false,false,false,false]
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    # Hk = H(xk)
    Hk = hess(nlp,xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    compteur_λ = 0 
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      if compteur_λ < 5 
        λ = max(1.0e-3, 10 * λ)
        dk = - ((Hk + λ * I ) \ gk)
        slope = dot(dk, gk)
        compteur_λ += 1 
      else
        dk = - gk
        critereArreteAtteint[1] = true # limite sur le nombre d'itérations 
        break  
      end
    end
    t = armijo(xk, dk, fk, gk, x->obj(nlp,x))
    xk += t * dk
    # fk = f(xk)
    fk = obj(nlp,xk)

    if neval_obj(nlp) >= 1000 
      critereArreteAtteint[3] = # critère d'arrêt sur le nombre d'évaluation de la fonction 
      break 
    end 

    # Arrêter la boucle si fk trop petite
    if fk < -1e15 
      critereArreteAtteint[4] = true # problème non-borné 
      break 
    end
    # gk = g(xk)
    gk = grad(nlp,xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
    t2 = time()
  
    if t2 - t1 > 60.0 
      critereArreteAtteint[2] = true # critère d'arrêt sur le temps 
      break 
    end

  end
  return xk
  if critereArreteAtteint[2]
    println("L'algorithme n'a pas trouvé de solution, il s'est arrêté à cause de la limite sur le temps d'exécution.")
  elseif critereArreteAtteint[3]
    println("L'algorithme n'a pas trouvé de solution, il s'est arrêté à cause de la limite sur le nombre d'évaluation de la fonction objectif.")
  elseif critereArreteAtteint[4]
    println("L'algorithme n'a pas trouvé de solution, il s'est arrêté car le problème est non borné.")
  end
end

newton_armijo (generic function with 1 method)

In [32]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0 = ones(2)
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  5.00e+00   1.2e+01
 1  4.07e-01   2.7e+00 1.0e+00 
 2  1.39e-02   4.3e-01 1.0e+00 
 3  4.63e-05   2.4e-02 1.0e+00 
 4  6.99e-10   9.2e-05 1.0e+00 
 5  1.63e-19   1.4e-09 1.0e+00 


2-element Vector{Float64}:
 2.3283064370807974e-10
 2.3283064370807974e-10

In [20]:
# Tests secrets 1: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.
using Test

@testset "Test set for newton_armijo" begin
    #Test problem:
    fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
    x0H = [10., 20.]
    himmelblau = ADNLPModel(fH, x0H)

    problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

    roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
    rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

    f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
    pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]


    ######################################### newton_armijo ##################

    #=
    Vérifiez que vous pouvez mettre une limite de temps de 1s.
    Puis faites tournez votre algorithme sur un des probléme test.
    L'algorithme devrait s'arrêter en environ 1sec.

    Vérifiez que vous pouvez mettre une limite de 2 évaluations d'objectif.
    Puis faites tournez votre algorithme sur un des probléme test.
    L'algorithme devrait s'arrêter en environ 1 itération.
    =#

    #Unit/Validation Tests
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
    sol = newton_armijo(himmelblau, himmelblau.meta.x0, verbose = false)
    @test norm(grad(himmelblau, sol)) ≤ ep1
    @test sol ≈ [3, 2] atol = ep1

    ep2 = 1e-6 + norm(grad(problem2, problem2.meta.x0)) * 1e-6
    sol =  newton_armijo(problem2, problem2.meta.x0, verbose = false)
    # @test stats.status == :unbounded
    @test obj(problem2, sol) ≤ -1e15

    ep2 = 1e-6 + norm(grad(rosenbrock, rosenbrock.meta.x0)) * 1e-6
    sol = newton_armijo(rosenbrock, rosenbrock.meta.x0, verbose = false)
    @test sol ≈ [1., 1.] atol = ep2

    ep3 = 1e-6 + norm(grad(pb_du_cours, [-1.001, -1.001])) * 1e-6
    sol = newton_armijo(pb_du_cours, [-1.001, -1.001], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep3 || obj(pb_du_cours, sol) <= -1e15

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
    sol = newton_armijo(pb_du_cours, [1.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep4 || obj(pb_du_cours, sol) <= -1e15

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
    sol = newton_armijo(pb_du_cours, [.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep5 || obj(pb_du_cours, sol) <= -1e15
end

Test Summary:              | Pass  Total  Time
Test set for newton_armijo |    7      7  1.9s


Test.DefaultTestSet("Test set for newton_armijo", Any[], 7, false, false, true, 1.674489895003154e9, 1.674489896861178e9)

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$
Ce type de factorisation n'est possible que si la matrice hessienne est définie positive, dans le cas contraire on a besoin de régularisé le système comme dans l'exercice précédent.

Pour résoudre le système linéaire en utilisant cette factorisation, on va utiliser le package [`LDLFactorizations`](https://github.com/JuliaSmoothOptimizers/LDLFactorizations.jl):

In [22]:
import Pkg; Pkg.add("LDLFactorizations")
using LDLFactorizations, LinearAlgebra

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


    Updating `~/Desktop/MTH8408/Project.toml`
  [40e66cde] + LDLFactorizations v0.10.0
  No Changes to `~/Desktop/MTH8408/Manifest.toml`


Un tutoriel sur l'utilisation de `LDLFactorizations` est disponible sur la documentation du package sur github ou encore [à ce lien](https://juliasmoothoptimizers.github.io/LDLFactorizations.jl/dev/tutorial/).

Voici un exemple d'utilisation de ce package. La matrice dont on veut calculer la factorisation doit être de type `Symmetric`.

In [ ]:
A = ones(2,2) #cette matrice symétrique, mais pas du type Symmetric
              #à noter que cette matrice n'est pas définie positive.
typeof(A) <: Symmetric #false
A = Symmetric(A)
typeof(A) <: Symmetric #true :)

Deuxième étape, le package fait une phase d'analyse de la matrice avec `ldl_analyze` en créant une structure pratique pour les diverses fonctions du package.

In [28]:
A = -rand(2, 2)
sol = rand(2)
b = A*sol #on veut résoudre le système A*x=b

# LDLFactorizations va en réalité demander la matrice triangulaire supérieure
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)
x = S \ b # x = A \b ça va être résolu par Julia 
norm(A * x - b)

4.965068306494546e-16

In [29]:
A = [0. 1.; 1. 0.]

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [30]:
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)

LDLFactorizations.LDLFactorization{Float64, Int64, Int64, Int64}(true, false, true, 2, [2, -1], [0, 0], [1, 2], [1, 2], [1, 2], [1, 2, 2], [1, 1, 1], Int64[], [4764090000], [3.0453972104e-314], [0.0, 2.354564522e-314], [0.0, 5.0e-324], [4766933904, 4766933920], 0.0, 0.0, 0.0, 2)

In [33]:
S.d 

2-element Vector{Float64}:
 0.0
 2.354564522e-314

La matrice `A` factorisée par $LDL^T$ n'était pas forcément définie positive. On peut le voir sur les valeurs de $D$.

In [38]:
S.d #c'est le vecteur qui correspond à la matrice diagonale D.

false

Pour l'optimisation, dans le cas où des valeurs de $D$ sont négatives, i.e. `minimum(S.d) <= 0.`, on ajoutera une correction pour être sûr d'obtenir une direction de descente. On pourra choisir un des deux:
- `S.d   = abs.(S.d)`
- `S.d .+= -minimum(S.d) + 1e-6`

#### Utiliser cette technique pour calculer la direction de descente:

In [47]:
# Solution: modifier le calcul de la direction avec LDLFactorizations
function newton_ldlt_armijo(nlp, x0; verbose::Bool = true)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = Symmetric(triu(hess(nlp, xk)), :U)
    # ... TODO ...
    Sk = ldl_analyze(Hk)
    ldl_factorize!(Hk, Sk)
    if !all(Sk.d .>= 0) 
      Sk.d .+= -minimum(Sk.d) + 1e-6
    end
    dk = - Sk \ gk
    slope = dot(dk, gk)
    t = armijo(xk, dk, fk, gk, x -> obj(nlp, x))
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_ldlt_armijo (generic function with 1 method)

In [45]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))

newton_ldlt_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  5.00e+00   1.2e+01
 1  4.07e-01   2.7e+00 1.0e+00 
 2  1.39e-02   4.3e-01 1.0e+00 
 3  4.63e-05   2.4e-02 1.0e+00 
 4  6.99e-10   9.2e-05 1.0e+00 
 5  1.63e-19   1.4e-09 1.0e+00 


2-element Vector{Float64}:
 2.3283064370807974e-10
 2.3283064370807974e-10

In [48]:
# Tests secrets 2: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.

using Test

@testset "Test set for newton_ldlt_armijo" begin
    #Test problem:
    fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
    x0H = [10., 20.]
    himmelblau = ADNLPModel(fH, x0H)

    problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

    roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
    rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

    f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
    pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]

    #Unit/Validation Tests
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(himmelblau, himmelblau.meta.x0, verbose = false)
    @test norm(grad(himmelblau, sol)) ≤ ep1
    @test sol ≈ [3, 2] atol = ep1

    ep2 = 1e-6 + norm(grad(problem2, problem2.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(problem2, problem2.meta.x0, verbose = false)
    # @test stats.status == :unbounded
    @test obj(problem2, sol) ≤ -1e15

    ep2 = 1e-6 + norm(grad(rosenbrock, rosenbrock.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(rosenbrock, rosenbrock.meta.x0, verbose = false)
    @test sol ≈ [1., 1.] atol = ep2

    ep3 = 1e-6 + norm(grad(pb_du_cours, [-1.001, -1.001])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [-1.001, -1.001], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep3 || obj(pb_du_cours, sol) <= -1e15

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [1.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep4 || obj(pb_du_cours, sol) <= -1e15

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep5 || obj(pb_du_cours, sol) <= -1e15
end

Test Summary:                   | Pass  Total  Time
Test set for newton_ldlt_armijo |    7      7  1.5s


Test.DefaultTestSet("Test set for newton_ldlt_armijo", Any[], 7, false, false, true, 1.674491773364915e9, 1.674491774865108e9)

### Exercice 3: Méthode quasi-Newton: BFGS

#### Méthode quasi-Newton: BFGS
Pour des problèmes de très grandes tailles, il est parfois très coûteux d'évaluer la hessienne du problème d'optimisation (et même le produit hessienne-vecteur). La famille des méthode *quasi-Newton* construit une approximation $B_k$ symétrique définie positive de la matrice Hessienne en utilisant seulement le gradient et en mesurant sa variation, et permet quand même d'améliorer significativement les performances comparé à la méthode du gradient.
$$
s_k = x_{k+1} - x_k, \quad y_k = \nabla f(x_{k+1}) - \nabla f(x_k).
$$
Par ailleurs la matrice $B_k$ est aussi construite de façon à ce que l'inverse soit connue, il n'y a donc pas de système linéaire à résoudre.

La méthode la plus connue dans la famille des méthodes quasi-Newton, est la méthode BFGS (Broyden - Fletcher, Goldfarb, and Shanno), et utilise la formule suivante pour calculer l'inverse de $B_k$ que l'on note $H_k$:
$$
H_{k+1} = (I - \rho_k s_ky_k^T)H_k(I-\rho_ky_ks_k^T) + \rho_ks_ks_k^T, \quad \rho_k = \frac{1}{y_k^Ts_k}.
$$
L'algorithme est presque le même que la méthode de Newton à la différence qu'il n'y a pas de système linéaire à résoudre et la direction $d_k$ est à coup sûr une direction de descente. Ainsi la direction de descente est calculée comme suit:
$$
d_k = - H_k \nabla f(x_k).
$$

Comment choisir la matrice $H_0$? On peut éventuellement choisir $I$. Une alternative est d'utiliser $H_0=I$ pour la première itération et ensuite mettre $H_0$ à jour avant de calculer $H_1$ en utilisant:
$$
H_0 = \frac{y_k^Ts_k}{y_k^Ty_k}I.
$$

**Important**: pour s'assurer que la matrice $H_k$ reste définie positive à toutes les itérations, il faut s'assurer que $y_k^Ts_k>0$. C'est toujours vrai pour des fonctions convexes, mais pas nécessairement dans le cas général. On pourra tester ici la version "skip" qui ne mets pas à jour quand cette condition n'est pas vérifiée.

In [22]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec la méthode de BFGS inverse skip.
function bfgs_quasi_newton_armijo(nlp, x0; verbose::Bool = true)
  xk = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)  # gnorm0 = norm(gk)
  k = 0
  t=0
  verbose && @printf "%2s %9s %9s %7s\n" "k" "fk" "||∇f(x)||" "t"
  verbose && @printf "%2d %9.2e %9.1e %7.1e\n" k fk gnorm t

  # première itération
  H0 = I(length(xk))
  d0 = - H0*gk
  t = armijo(xk, d0, fk, gk, x -> obj(nlp, x))
  xk_1 = xk + t * d0
  fk = obj(nlp, xk_1)
  gk = grad(nlp, xk_1)
  gnorm = norm(gk)
  k += 1
  verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t

  yk = grad(nlp,xk_1) - grad(nlp,xk)
  sk = xk_1 - xk 
  ρk = 1/(transpose(yk)*sk)
  Hk = (transpose(yk)*sk)/(transpose(yk)*yk)*I(length(xk_1))
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    if transpose(yk)*sk > 0 
      Hk_1 = (I(length(xk_1)) - ρk*sk*transpose(yk))* Hk * (I(length(xk_1)) - ρk*yk*transpose(sk)) + ρk*sk*transpose(sk)
    else 
      Hk_1 = Hk
    end
    dk = - Hk_1 *grad(nlp,xk_1)
    xk = xk_1
    fk = obj(nlp,xk_1)
    gk = grad(nlp, xk_1)
    t = armijo(xk_1, dk, fk, gk, x -> obj(nlp, x))
    xk_1 += t * dk 
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
    yk = grad(nlp,xk_1) - grad(nlp,xk)
    sk = xk_1 - xk 
    ρk = 1/(transpose(yk)*sk)
  end
  return xk
end

bfgs_quasi_newton_armijo (generic function with 1 method)

In [23]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))
x0 = zeros(2)
bfgs_quasi_newton_armijo(nlp, x0)

 k        fk ||∇f(x)||       t
 0  0.00e+00   0.0e+00 0.0e+00
 1  0.00e+00   0.0e+00 1.0e+00 


2-element Vector{Float64}:
 0.0
 0.0

In [24]:
# Tests secrets 3: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.
using Test

@testset "Test set for bfgs_quasi_newton_armijo" begin
	#Test problem:
	fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
	x0H = [10., 20.]
	himmelblau = ADNLPModel(fH, x0H)

	problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

	roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
	rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

	f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
	pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]


	######################################### bfgs_quasi_newton_armijo ##################

	#Unit/Validation Tests
	using Logging, Test
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
	sol = bfgs_quasi_newton_armijo(himmelblau, himmelblau.meta.x0) 
	@test norm(grad(himmelblau, sol)) ≤ ep1

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
	sol = bfgs_quasi_newton_armijo(pb_du_cours, [1.5, .5]) 
	@test norm(grad(pb_du_cours, sol)) ≤ ep4
	@test sol ≈ [1, 0] atol = ep4
	@show sol

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
	sol = bfgs_quasi_newton_armijo(pb_du_cours, [.5, .5]) 
	@test norm(grad(pb_du_cours, sol)) ≤ ep4
	@test sol ≈ [1, 0] atol = ep4
	@show sol
end

 k        fk ||∇f(x)||       t
 0  1.74e+05   3.3e+04 0.0e+00
 1  2.73e+04   8.6e+03 1.0e-03 
 2  2.73e+04   8.6e+03 1.0e+00 
 3  1.26e+03   8.6e+02 1.0e+00 
 4  7.23e+02   5.8e+02 1.0e+00 
 5  1.70e+02   2.1e+02 1.0e+00 
 6  5.22e+01   9.6e+01 1.0e+00 
 7  1.18e+01   4.1e+01 1.0e+00 
 8  2.14e+00   1.8e+01 1.0e+00 
 9  3.01e-01   7.7e+00 1.0e+00 
10  1.05e-01   4.6e+00 1.0e+00 
11  8.39e-02   3.7e+00 1.0e+00 
12  7.10e-02   3.0e+00 1.0e+00 
13  6.29e-02   2.5e+00 1.0e+00 
14  5.72e-02   2.0e+00 1.0e+00 
15  5.32e-02   1.8e+00 1.0e+00 
16  4.97e-02   1.6e+00 1.0e+00 
17  4.66e-02   1.6e+00 1.0e+00 
18  4.31e-02   1.7e+00 1.0e+00 
19  3.93e-02   1.8e+00 1.0e+00 
20  3.47e-02   1.9e+00 1.0e+00 
21  2.96e-02   2.0e+00 1.0e+00 
22  2.40e-02   2.0e+00 1.0e+00 
23  1.89e-02   2.0e+00 1.0e+00 
24  1.46e-02   1.7e+00 1.0e+00 
25  1.16e-02   1.5e+00 1.0e+00 
26  9.41e-03   1.2e+00 1.0e+00 
27  8.04e-03   1.0e+00 1.0e+00 
28  7.10e-03   8.2e-01 1.0e+00 
29  6.47e-03   7.0e-01 1.0e+00 
30  5.97e-

 k        fk ||∇f(x)||       t
 0  0.00e+00   4.5e+00 0.0e+00
 1 -3.73e-01   4.2e+00 8.8e-02 
 2 -3.73e-01   4.2e+00 1.0e+00 
 3 -7.22e-01   1.4e+00 1.0e+00 
 4 -8.07e-01   1.1e+00 1.0e+00 
 5 -9.46e-01   7.8e-01 1.0e+00 
 6 -9.96e-01   2.8e-01 1.0e+00 
 7 -1.00e+00   7.2e-02 1.0e+00 
 8 -1.00e+00   2.1e-02 1.0e+00 
 9 -1.00e+00   9.9e-03 1.0e+00 
10 -1.00e+00   5.2e-03 1.0e+00 
11 -1.00e+00   4.9e-03 1.0e+00 
12 -1.00e+00   4.6e-03 1.0e+00 
13 -1.00e+00   1.8e-03 1.0e+00 
14 -1.00e+00   4.8e-04 1.0e+00 
15 -1.00e+00   2.1e-04 1.0e+00 
16 -1.00e+00   1.1e-04 1.0e+00 
17 -1.00e+00   9.4e-05 1.0e+00 
18 -1.00e+00   9.3e-05 1.0e+00 
19 -1.00e+00   4.3e-05 1.0e+00 
20 -1.00e+00   1.1e-05 1.0e+00 
21 -1.00e+00   4.7e-06 1.0e+00 
sol = [0.9999990683502704, -2.6409525493569896e-7]
 k        fk ||∇f(x)||       t
 0  1.00e+00   4.5e+00 0.0e+00
 1  3.33e-01   4.4e+00 3.0e-01 
 2  3.33e-01   4.4e+00 1.0e+00 
 3 -4.00e-01   1.4e+00 1.0e+00 
 4 -5.80e-01   1.3e+00 6.7e-01 
 5 -8.63e-01   1.8e+00 1.

Test.DefaultTestSet("Test set for bfgs_quasi_newton_armijo", Any[], 5, false, false, true, 1.675178242864879e9, 1.67517824332894e9)

### Exercice 4: application à un problème de grande taille

On va ajouter le package `OptimizationProblems` qui contient, comme son nom l'indique, une collection de problème d'optimisation disponible au format de `JuMP` (dans le sous-module `OptimizationProblems.PureJuMP`) et de `ADNLPModel` (dans le sous-module `OptimizationProblems.ADNLPProblems`).

In [ ]:
import Pkg; Pkg.add("OptimizationProblems")

In [6]:
using ADNLPModels, OptimizationProblems.ADNLPProblems # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!

In [ ]:
n = 500
model = genrose(n=n)
@test typeof(model) <: ADNLPModel

Si vous le souhaitez, il est possible d'accéder à certaines informations sur le problème en accédant à son meta:

In [ ]:
using OptimizationProblems
OptimizationProblems.genrose_meta

Il est aussi possible d'accéder au meta de tous les problèmes

In [ ]:
OptimizationProblems.meta

Résoudre le problème `genrose` et un autre problème de la collection en utilisant vos algorithmes précédents.
Avant d'utiliser l'algorithme on testera que le problème est bien sans contrainte avec:

In [ ]:
unconstrained(nlp) #qui retourne vrai si `nlp` est un problème sans contraintes.

In [ ]:
# Use previous functions to solve genrose.